In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, tool, AgentExecutor, AgentType, initialize_agent, load_tools

import os
from dotenv import load_dotenv

load_dotenv()
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
search = TavilySearchResults()

In [2]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini-2024-07-18")
tools = [search]
message_history = ChatMessageHistory()

In [3]:
def create_markdown_report(response, filename):
    markdown_content = response['output']

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        return True
    except Exception as e:
        print(f"Error creating file: {str(e)}")
        return False

### Search Agent

In [4]:
search_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Think step by step before responding."), # For initial persona or instruction
        ("placeholder", "{chat_history}"), # For historical context such as previous response
        ("human", "{input}"), # User query
        ("placeholder", "{agent_scratchpad}") # For previous agent response and tools
    ]
)

In [5]:
search_calling_agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=search_prompt)
search_agent_executor = AgentExecutor(agent=search_calling_agent, tools=tools, verbose=True)

In [6]:
search_agent = RunnableWithMessageHistory(
    search_agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [7]:
search_response = search_agent.invoke(
    {"input": "Give 3 research papers about Risk Communication"},
    config={"configurable": {"session_id": "search_session"}}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Risk Communication research papers'}`


[{'url': 'https://www.sra.org/risk-analysis-specialty-groups/risk-communication/research/', 'content': 'Login | Register Go To Search Page Risk Analysis Glossary Decision Analysis and Risk Risk Communication Archived Annual Meetings Best Paper & Reviewer Awards Risk Analysis Glossary Decision Analysis and Risk Risk Communication Archived Annual Meetings Best Paper & Reviewer Awards Risk Communication The journals below represent just a few of the many different types of risk communication research. Also, an RSS feed for SRA’s own journal – Risk Analysis – is also provided below. Journal of Risk Research Articles coming soon and main journal site –\xa0http://www.tandfonline.com/toc/rjrr20/current Articles coming soon and main journal site –\xa0http://www.tandfonline.com/toc/uhcm20/current Articles coming soon and main journal site –\xa0http://www.tandf

In [8]:
print(search_response['output'])

Here are three research papers related to Risk Communication:

1. **Risk Communication in Disaster Risk Reduction**  
   - **Link**: [ScienceDirect Article](https://www.sciencedirect.com/science/article/pii/S2590061720300764)  
   - **Summary**: This paper discusses the critical role of risk communication in disaster risk reduction, particularly in the design and implementation of early warning systems.

2. **Risk Communication and Risk Perception**  
   - **Link**: [BMC Public Health Article](https://bmcpublichealth.biomedcentral.com/articles/10.1186/s12889-020-10125-5)  
   - **Summary**: This study explores how risk communication and risk perception influence protective and preventive behaviors in the context of public health.

3. **Review of Risk Communication Research**  
   - **Link**: [Science.gov Review](https://www.science.gov/topicpages/r/risk+communication+research.html)  
   - **Summary**: This article reviews risk communication research published since 1996, focusing on as

### Summary Agent

In [9]:
report_prompt_template = """This report type focuses on comparative analysis.

The report structure should include:
1. Introduction (no research needed)
   - Brief overview of the topic area
   - Context for the comparison

2. Main Body Sections:
   - One dedicated section for EACH offering being compared in the user-provided list
   - Each section should examine:
     - Core Features (bulleted list)
     - Architecture & Implementation (2-3 sentences)
     - One example use case (2-3 sentences)
   
3. No Main Body Sections other than the ones dedicated to each offering in the user-provided list

4. Conclusion with Comparison Table (no research needed)
   - Structured comparison table that:
     * Compares all offerings from the user-provided list across key dimensions
     * Highlights relative strengths and weaknesses
   - Final recommendations
   
5. Sources
   - In a bulleted format, include the sources used
"""

In [10]:
research_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Think step by step before responding."), # For initial persona or instruction
        ("system", "{prompt_template}"),
        ("placeholder", "{chat_history}"), # For historical context such as previous response
        ("human", "{input}"), # User query
        ("placeholder", "{agent_scratchpad}") # For previous agent response and tools
    ]
)

In [11]:
research_calling_agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=research_prompt)
research_agent_executor = AgentExecutor(agent=research_calling_agent, tools=tools, verbose=True)

In [12]:
research_agent = RunnableWithMessageHistory(
    search_agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
research_response = research_agent.invoke(
    {"input": "From these research papers, write a report about Risk Communication",
     "prompt_template": research_prompt_template,
     },
    config={"configurable": {"session_id": "search_session"}}
)

NameError: name 'prompt_template' is not defined

In [18]:
print(research_response['output'])

### Report on Risk Communication

#### Introduction
Risk communication is a critical aspect of public health, environmental management, and emergency response. It involves the exchange of information about risks between decision-makers and the public, aiming to inform and empower individuals to make informed decisions. This report synthesizes findings from three significant research papers on risk communication, highlighting its evolution, methodologies, and practical applications.

#### Overview of Risk Communication
Risk communication has evolved significantly over the past few decades. It is no longer just about conveying information; it involves understanding the audience, their perceptions, and the context in which they receive information. Effective risk communication is essential in managing public concerns, especially during emergencies or crises.

#### Key Findings from Research Papers

1. **Proactive Response to Citizen Risk Concerns**
   - **Authors:** Heath, R. L., & Abel, 

In [20]:
create_markdown_report(research_response, filename="Summary.md")

True